In [61]:
import websocket, json, pprint, talib, numpy
import config
from binance.client import Client
from binance.enums import *
import math

In [62]:
SOCKET = "wss://stream.binance.com:9443/ws/zilusdt@kline_5m"

In [63]:
SMA_LOW = 2
SMA_HIGH = 7
TRADE_SYMBOL = 'ZILUSDT'
BASE ='USDT'
ALT='ZIL'

In [65]:
closes = []
in_position = True

In [66]:
client = Client(config.API_KEY, config.API_SECRET)

In [67]:
def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET,price='0.0001',timeInForce=TIME_IN_FORCE_GTC):
    try:
        print("sending order")
        if order_type == ORDER_TYPE_MARKET:
            order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        elif order_type == ORDER_TYPE_LIMIT:
            order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity,price=price,timeInForce=timeInForce)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

In [68]:
def on_open(ws):
    print('opened connection')

In [69]:
def on_close(ws):
    print('closed connection')

In [70]:
def on_message(ws, message):
    global closes, in_position
    
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)
    
    candle = json_message['k']
    
    is_candle_closed = candle['x']
    close = candle['c']
    
    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        #print(closes)
        print("The length of closes is {}".format(len(closes)))
        print("The value of SMA_HIGH is {}".format(SMA_HIGH))
        
        if len(closes) > SMA_HIGH:
            print("Step 1")
            np_closes = numpy.array(closes)
            SMA_H=talib.SMA(np_closes,timeperiod=SMA_HIGH)
            SMA_L=talib.SMA(np_closes,timeperiod=SMA_LOW)
            print("The value of SMA_H is {}".format(SMA_H))
            print("The value of SMA_L is {}".format(SMA_L))
            
            last_sma_h = SMA_H[-1]
            last_sma_l = SMA_L[-1]
            
            last_sma_l_2 = SMA_L[-2]
            last_sma_l_3 = SMA_L[-3]
            
            last_sma_h_2 = SMA_H[-2]
            last_sma_h_3 = SMA_H[-3]
            
            print(last_sma_h)
            print(last_sma_l)
            
            if last_sma_l < last_sma_h and last_sma_l_2 < last_sma_h_2 and last_sma_l_3 < last_sma_h_3:
                if in_position:
                    print("Nothing to Do, Already Bought")
                else:
                    if last_sma_l > last_sma_l_2 and last_sma_l_2 < last_sma_l_3:
                        TRADE_QUANTITY=float(client.get_asset_balance(asset=BASE)['free'])//np_closes[-1]
                        order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = True
                            print("BUY Order Placed and succeded")
                        else:
                            print("BUY Order placed but failed")
                        
                        
            if last_sma_l > last_sma_h and last_sma_l_2 > last_sma_h_2 and last_sma_l_3 > last_sma_h_3:
                if in_position:
                    if last_sma_l < last_sma_l_2 and last_sma_l_2 > last_sma_l_3:
                        TRADE_QUANTITY=math.floor(float(client.get_asset_balance(asset=ALT)['free'])*10)/10
                        order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = False
                            print("SELL ORDER Placed and succeeded")
                        else:
                            print("SELL Order placed but failed")
                else:
                    print('Do Nothing','Already Sold')

In [71]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
candle closed at 0.12352000
closes
The length of closes is 1
The value of SMA_HIGH is 7
candle closed at 0.12281000
closes
The length of closes is 2
The value of SMA_HIGH is 7
candle closed at 0.12271000
closes
The length of closes is 3
The value of SMA_HIGH is 7
candle closed at 0.12271000
closes
The length of closes is 4
The value of SMA_HIGH is 7
candle closed at 0.12340000
closes
The length of closes is 5
The value of SMA_HIGH is 7
candle closed at 0.12296000
closes
The length of closes is 6
The value of SMA_HIGH is 7
candle closed at 0.12117000
closes
The length of closes is 7
The value of SMA_HIGH is 7
candle closed at 0.11956000
closes
The length of closes is 8
The value of SMA_HIGH is 7
Step 1
The value of SMA_H is [       nan        nan        nan        nan        nan        nan
 0.12275429 0.12218857]
The value of SMA_L is [     nan 0.123165 0.12276  0.12271  0.123055 0.12318  0.122065 0.120365]
0.12218857142857142
0.12036499999999997
candle closed at 0.119

candle closed at 0.12029000
closes
The length of closes is 22
The value of SMA_HIGH is 7
Step 1
The value of SMA_H is [       nan        nan        nan        nan        nan        nan
 0.12275429 0.12218857 0.12172857 0.12144286 0.12133286 0.12126
 0.12116    0.12139    0.12196286 0.12247    0.12265571 0.12232286
 0.12190571 0.12151571 0.12121857 0.12075   ]
The value of SMA_L is [     nan 0.123165 0.12276  0.12271  0.123055 0.12318  0.122065 0.120365
 0.119575 0.12015  0.121325 0.122415 0.122575 0.12252  0.123175 0.123355
 0.122575 0.12081  0.11979  0.11975  0.120115 0.120495]
0.12075000000000002
0.12049499999999999
Nothing to Do, Already Bought
candle closed at 0.12067000
closes
The length of closes is 23
The value of SMA_HIGH is 7
Step 1
The value of SMA_H is [       nan        nan        nan        nan        nan        nan
 0.12275429 0.12218857 0.12172857 0.12144286 0.12133286 0.12126
 0.12116    0.12139    0.12196286 0.12247    0.12265571 0.12232286
 0.12190571 0.12151571 0.121

candle closed at 0.11837000
closes
The length of closes is 33
The value of SMA_HIGH is 7
Step 1
The value of SMA_H is [       nan        nan        nan        nan        nan        nan
 0.12275429 0.12218857 0.12172857 0.12144286 0.12133286 0.12126
 0.12116    0.12139    0.12196286 0.12247    0.12265571 0.12232286
 0.12190571 0.12151571 0.12121857 0.12075    0.12039714 0.12030714
 0.12047571 0.12052143 0.12047714 0.12025571 0.12051714 0.12055
 0.12021857 0.12000143 0.11972714]
The value of SMA_L is [     nan 0.123165 0.12276  0.12271  0.123055 0.12318  0.122065 0.120365
 0.119575 0.12015  0.121325 0.122415 0.122575 0.12252  0.123175 0.123355
 0.122575 0.12081  0.11979  0.11975  0.120115 0.120495 0.12048  0.121025
 0.121085 0.12054  0.119755 0.119185 0.120635 0.12151  0.11998  0.119165
 0.11882 ]
0.11972714285714284
0.11882000000000001
candle closed at 0.11724000
closes
The length of closes is 34
The value of SMA_HIGH is 7
Step 1
The value of SMA_H is [       nan        nan        nan  

candle closed at 0.11716000
closes
The length of closes is 42
The value of SMA_HIGH is 7
Step 1
The value of SMA_H is [       nan        nan        nan        nan        nan        nan
 0.12275429 0.12218857 0.12172857 0.12144286 0.12133286 0.12126
 0.12116    0.12139    0.12196286 0.12247    0.12265571 0.12232286
 0.12190571 0.12151571 0.12121857 0.12075    0.12039714 0.12030714
 0.12047571 0.12052143 0.12047714 0.12025571 0.12051714 0.12055
 0.12021857 0.12000143 0.11972714 0.11944429 0.11920429 0.11863857
 0.11817714 0.1178     0.11755714 0.11726857 0.11708286 0.11703857]
The value of SMA_L is [     nan 0.123165 0.12276  0.12271  0.123055 0.12318  0.122065 0.120365
 0.119575 0.12015  0.121325 0.122415 0.122575 0.12252  0.123175 0.123355
 0.122575 0.12081  0.11979  0.11975  0.120115 0.120495 0.12048  0.121025
 0.121085 0.12054  0.119755 0.119185 0.120635 0.12151  0.11998  0.119165
 0.11882  0.117805 0.117355 0.117815 0.117915 0.117045 0.116995 0.11696
 0.116145 0.11655 ]
0.1170385714

False

In [31]:
math.floor(float(client.get_asset_balance(asset=ALT)['free'])*10)/10

58.9

In [32]:
TRADE_QUANTITY=math.floor(float(client.get_asset_balance(asset=ALT)['free'])*10)/10

In [42]:
order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL,ORDER_TYPE_LIMIT,price='0.57')

sending order
{'symbol': 'XRPUSDT', 'orderId': 1735352129, 'orderListId': -1, 'clientOrderId': 'kMRqt0kg8QmTXOdGMBC5qU', 'transactTime': 1613814820156, 'price': '0.57000000', 'origQty': '58.90000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'SELL', 'fills': []}


In [4]:
candlesticks = client.get_historical_klines("ZILUSDT",Client.KLINE_INTERVAL_15MINUTE,"1 day ago UTC")

In [6]:
closes=[]

In [7]:
for candle in candlesticks:
    closes.append(float(candle[4]))
print(closes)

[0.13977, 0.13929, 0.1391, 0.139, 0.13951, 0.13942, 0.13917, 0.13942, 0.1395, 0.13969, 0.13948, 0.13948, 0.13946, 0.13925, 0.13967, 0.13967, 0.13981, 0.13968, 0.13952, 0.13959, 0.14001, 0.13994, 0.13993, 0.13974, 0.14007, 0.14003, 0.14002, 0.14009, 0.13996, 0.13967, 0.13977, 0.13948, 0.13955, 0.13828, 0.13832, 0.13812, 0.13777, 0.13748, 0.13775, 0.13803, 0.13833, 0.13798, 0.13764, 0.13724, 0.13721, 0.13779, 0.13782, 0.13751, 0.1373, 0.13761, 0.1379, 0.13784, 0.13799, 0.13794, 0.13862, 0.1383, 0.13846, 0.13823, 0.13833, 0.13872, 0.13858, 0.13866, 0.13838, 0.13838, 0.13898, 0.13908, 0.13903, 0.13886, 0.13905, 0.13919, 0.13958, 0.13955, 0.13961, 0.13948, 0.13955, 0.13953, 0.13934, 0.13931, 0.13941, 0.13939, 0.13959, 0.1396, 0.13983, 0.13935, 0.13974, 0.13967, 0.13973, 0.14, 0.13961, 0.13959, 0.1398, 0.13985, 0.13967, 0.13962, 0.13928, 0.13947, 0.1394, 0.13951, 0.1395, 0.13978, 0.1399, 0.13963, 0.13965, 0.1399, 0.13986, 0.13926, 0.13921, 0.13911, 0.13932, 0.13955, 0.13938, 0.13911, 0.13912

In [8]:
del closes[-1]

In [9]:
print(closes)

[0.13977, 0.13929, 0.1391, 0.139, 0.13951, 0.13942, 0.13917, 0.13942, 0.1395, 0.13969, 0.13948, 0.13948, 0.13946, 0.13925, 0.13967, 0.13967, 0.13981, 0.13968, 0.13952, 0.13959, 0.14001, 0.13994, 0.13993, 0.13974, 0.14007, 0.14003, 0.14002, 0.14009, 0.13996, 0.13967, 0.13977, 0.13948, 0.13955, 0.13828, 0.13832, 0.13812, 0.13777, 0.13748, 0.13775, 0.13803, 0.13833, 0.13798, 0.13764, 0.13724, 0.13721, 0.13779, 0.13782, 0.13751, 0.1373, 0.13761, 0.1379, 0.13784, 0.13799, 0.13794, 0.13862, 0.1383, 0.13846, 0.13823, 0.13833, 0.13872, 0.13858, 0.13866, 0.13838, 0.13838, 0.13898, 0.13908, 0.13903, 0.13886, 0.13905, 0.13919, 0.13958, 0.13955, 0.13961, 0.13948, 0.13955, 0.13953, 0.13934, 0.13931, 0.13941, 0.13939, 0.13959, 0.1396, 0.13983, 0.13935, 0.13974, 0.13967, 0.13973, 0.14, 0.13961, 0.13959, 0.1398, 0.13985, 0.13967, 0.13962, 0.13928, 0.13947, 0.1394, 0.13951, 0.1395, 0.13978, 0.1399, 0.13963, 0.13965, 0.1399, 0.13986, 0.13926, 0.13921, 0.13911, 0.13932, 0.13955, 0.13938, 0.13911, 0.13912